In [1]:
import kagglehub
path = kagglehub.dataset_download("safabouguezzi/german-traffic-sign-detection-benchmark-gtsdb")
print("Path to dataset files:", path)

100%|██████████| 1.61G/1.61G [00:18<00:00, 94.3MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/safabouguezzi/german-traffic-sign-detection-benchmark-gtsdb/versions/1


In [ ]:
import os
import shutil
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split

# Paths
source_root = '/root/.cache/kagglehub/datasets/safabouguezzi/german-traffic-sign-detection-benchmark-gtsdb/versions/1'
image_dir = os.path.join(source_root, 'TrainIJCNN2013', 'TrainIJCNN2013')
gt_path = os.path.join(source_root, 'gt.txt')
dest_root = '/content/GTSDB-YOLO'
splits = ['train', 'val', 'test']

img_dirs = {s: os.path.join(dest_root, 'images', s) for s in splits}
lbl_dirs = {s: os.path.join(dest_root, 'labels', s) for s in splits}

for d in list(img_dirs.values()) + list(lbl_dirs.values()):
    os.makedirs(d, exist_ok=True)


df = pd.read_csv(gt_path, sep=';', header=None)
df.columns = ['filename', 'x_min', 'y_min', 'x_max', 'y_max', 'class_id']


all_images = df['filename'].unique()


train_files, temp_files = train_test_split(
    all_images,
    test_size=0.3,
    random_state=42
)


val_files, test_files = train_test_split(
    temp_files,
    test_size=0.5,
    random_state=42
)

def convert_and_save_yolo_labels(image_set, img_out_dir, lbl_out_dir):
    for filename in image_set:
        img_path = os.path.join(image_dir, filename)
        lbl_path = os.path.join(lbl_out_dir, filename.replace('.ppm', '.txt'))

        if not os.path.exists(img_path):
            continue

        shutil.copy(img_path, os.path.join(img_out_dir, filename))

        with Image.open(img_path) as img:
            w, h = img.size

        boxes = df[df['filename'] == filename]
        yolo_lines = []
        for _, row in boxes.iterrows():
            x_min, y_min, x_max, y_max = row[['x_min', 'y_min', 'x_max', 'y_max']]
            class_id = row['class_id']

            x_center = (x_min + x_max) / 2.0 / w
            y_center = (y_min + y_max) / 2.0 / h
            width = (x_max - x_min) / w
            height = (y_max - y_min) / h

            yolo_lines.append(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

        with open(lbl_path, 'w') as f:
            f.write('\n'.join(yolo_lines))


convert_and_save_yolo_labels(train_files, img_dirs['train'], lbl_dirs['train'])
convert_and_save_yolo_labels(val_files, img_dirs['val'], lbl_dirs['val'])
convert_and_save_yolo_labels(test_files, img_dirs['test'], lbl_dirs['test'])


for split in splits:
    img_dir = img_dirs[split]
    lbl_dir = lbl_dirs[split]

    for file in os.listdir(img_dir):
        if file.endswith('.ppm'):
            old_img_path = os.path.join(img_dir, file)
            new_img_name = file.replace('.ppm', '.jpg')
            new_img_path = os.path.join(img_dir, new_img_name)

            with Image.open(old_img_path) as im:
                rgb_im = im.convert('RGB')
                rgb_im.save(new_img_path, 'JPEG')
            os.remove(old_img_path)

            old_lbl = os.path.join(lbl_dir, file.replace('.ppm', '.txt'))
            new_lbl = os.path.join(lbl_dir, new_img_name.replace('.jpg', '.txt'))
            if os.path.exists(old_lbl):
                os.rename(old_lbl, new_lbl)


class_names = [
    "speed limit 20 (prohibitory)", "speed limit 30 (prohibitory)", "speed limit 50 (prohibitory)",
    "speed limit 60 (prohibitory)", "speed limit 70 (prohibitory)", "speed limit 80 (prohibitory)",
    "restriction ends 80 (other)", "speed limit 100 (prohibitory)", "speed limit 120 (prohibitory)",
    "no overtaking (prohibitory)", "no overtaking (trucks) (prohibitory)", "priority at next intersection (danger)",
    "priority road (other)", "give way (other)", "stop (other)", "no traffic both ways (prohibitory)",
    "no trucks (prohibitory)", "no entry (other)", "danger (danger)", "bend left (danger)",
    "bend right (danger)", "bend (danger)", "uneven road (danger)", "slippery road (danger)",
    "road narrows (danger)", "construction (danger)", "traffic signal (danger)", "pedestrian crossing (danger)",
    "school crossing (danger)", "cycles crossing (danger)", "snow (danger)", "animals (danger)",
    "restriction ends (other)", "go right (mandatory)", "go left (mandatory)", "go straight (mandatory)",
    "go right or straight (mandatory)", "go left or straight (mandatory)", "keep right (mandatory)",
    "keep left (mandatory)", "roundabout (mandatory)", "restriction ends (overtaking) (other)",
    "restriction ends (overtaking (trucks)) (other)"
]

dataset_yaml_path = os.path.join(dest_root, "dataset.yaml")

yaml_content = f"""train: {img_dirs['train']}
val: {img_dirs['val']}
test: {img_dirs['test']}

nc: 43
names: {class_names}
"""

with open(dataset_yaml_path, "w") as f:
    f.write(yaml_content)


In [5]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 66.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8n.yaml')


model.train(
    data='/content/GTSDB-YOLO/dataset.yaml',
    epochs=100,
    imgsz=640,
    batch=16,
    name='gtsdb_yolov8n',
    augment = False,
    optimizer = 'AdamW',
    patience=10,
    verbose=True,
    plots=False,
)


Ultralytics 8.3.161 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/GTSDB-YOLO/dataset.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.yaml, momentum=0.937, mosaic=1.0, multi_scale=False, name=gtsdb_yolov8n2, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=10, perspective=0.0, plots=False, pose

train: Scanning /content/GTSDB-YOLO/labels/train.cache... 354 images, 0 backgrounds, 0 corrupt: 100%|██████████| 354/354 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 1.4±3.1 ms, read: 669.7±479.3 MB/s, size: 82.1 KB)


val: Scanning /content/GTSDB-YOLO/labels/val.cache... 76 images, 0 backgrounds, 0 corrupt: 100%|██████████| 76/76 [00:00<?, ?it/s]

...

                   all         76        113      0.688       0.25      0.317      0.233
EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 68, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



78 epochs completed in 0.190 hours.
Optimizer stripped from runs/detect/gtsdb_yolov8n2/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/gtsdb_yolov8n2/weights/best.pt, 6.2MB

Validating runs/detect/gtsdb_yolov8n2/weights/best.pt...
Ultralytics 8.3.161 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8n summary (fused): 72 layers, 3,014,033 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:00<00:00,  5.95it/s]

                   all         76        113       0.61      0.293      0.317      0.236
speed limit 30 (prohibitory)          5          6      0.287      0.333      0.348      0.255
speed limit 50 (prohibitory)          9         10     0.0973        0.1      0.164      0.118
speed limit 60 (prohibitory)          1          1          1          0     0.0553     0.0553
speed limit 70 (prohibitory)          3          3          1          0     0.0467      0.036
speed limit 80 (prohibitory)          5          8       0.45      0.614      0.619      0.462
restriction ends 80 (other)          2          3      0.327      0.333      0.388     0.0974
speed limit 100 (prohibitory)          4          6      0.162        0.5      0.283      0.221
speed limit 120 (prohibitory)          1          1     0.0993          1      0.995      0.697
no overtaking (prohibitory)          4          5          1          0      0.176      0.114
no overtaking (trucks) (prohibitory)          1         

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 18, 19, 20, 21, 23, 25, 27, 28, 29, 30, 33, 34, 38])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7eed12715490>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004

In [21]:
model = YOLO('/content/runs/detect/gtsdb_yolov8n2/weights/best.pt')

metrics = model.val(
    data='/content/GTSDB-YOLO/dataset.yaml',
    split='test',
    plots=False
)

print(f"Precision: {metrics.box.p.mean():.4f}")
print(f"Recall: {metrics.box.r.mean():.4f}")
print(f"mAP@0.5: {metrics.box.map50:.4f}")
print(f"mAP@0.5:0.95: {metrics.box.map:.4f}")


Ultralytics 8.3.161 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8n summary (fused): 72 layers, 3,014,033 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2099.0±455.5 MB/s, size: 133.3 KB)


val: Scanning /content/GTSDB-YOLO/labels/test.cache... 76 images, 0 backgrounds, 0 corrupt: 100%|██████████| 76/76 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:00<00:00,  5.07it/s]


                   all         76        120      0.442      0.348      0.275      0.202
speed limit 20 (prohibitory)          1          1          1          0     0.0622      0.056
speed limit 30 (prohibitory)          9          9      0.504      0.667      0.608       0.48
speed limit 50 (prohibitory)          4          5      0.142        0.4      0.118      0.103
speed limit 60 (prohibitory)          3          3          1          0      0.226      0.181
speed limit 70 (prohibitory)          3          3          0          0     0.0115     0.0104
speed limit 80 (prohibitory)          3          5      0.179        0.6      0.331      0.271
speed limit 100 (prohibitory)          3          5      0.174        0.8      0.258      0.201
speed limit 120 (prohibitory)          5          8      0.204      0.875      0.462      0.305
no overtaking (prohibitory)          5          6      0.228      0.333      0.417      0.168
no overtaking (trucks) (prohibitory)         10        